In [1]:
#Preprocesamiento de información del Censo de Población y Vivienda

#Importar librerías
import pandas as pd
import os
import zipfile
import shutil

In [2]:
#Definir directorio de trabajo
#Directorio de trabajo. Si existe directorio D:/, se selecciona si no, C:/
if os.path.exists('D:/'):
    os.chdir('D:/github/proyecto_infotec')
else:
    os.chdir('C:/users/claud/Documents/Github/proyecto_infotec')

In [79]:
#crear directorio de salida
if not os.path.exists('datos'):
    os.makedirs('datos')

In [80]:
#URL básica para descarga de archivos
url_basica="https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_"

In [81]:
#Descargar archivos y descomprimir archivos

for entidad in range(1,33):
    #Si existe el archivo, no se descarga
    if os.path.exists("datos/conjunto_de_datos_ageb_urbana_01_cpv2020.csv"):
        pass
    else:
        if entidad<10:
            url=url_basica+"0"+str(entidad)+"_cpv2020_csv.zip"
        else:
            url=url_basica+str(entidad)+"_cpv2020_csv.zip"
        
#Descargar archivo
        os.system("curl -o datos/entidad_"+str(entidad)+".zip "+url)
#Descomprimir archivo
        with zipfile.ZipFile("datos/entidad_"+str(entidad)+".zip","r") as zip_ref:
            zip_ref.extractall("datos")
#Eliminar archivo zip
        os.remove("datos/entidad_"+str(entidad)+".zip")

In [85]:
#Sacar de la carpeta conjunto_de_datos el archivo csv de cada entidad
for entidad in range(1,33):
    if entidad<10:
        shutil.move("datos/ageb_mza_urbana_0"+str(entidad)+"_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_0"+str(entidad)+"_cpv2020.csv","datos/conjunto_de_datos_ageb_urbana_0"+str(entidad)+"_cpv2020.csv")
    else:
        shutil.move("datos/ageb_mza_urbana_"+str(entidad)+"_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv","datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv")

In [88]:
#Eliminar las carpetas del directorio de trabajo si existen
for entidad in range(1,33):
    if entidad<10:
        if os.path.exists("datos/ageb_mza_urbana_"+"0"+str(entidad)+"_cpv2020"):
            shutil.rmtree("datos/ageb_mza_urbana_"+"0"+str(entidad)+"_cpv2020")
    else:
        if os.path.exists("datos/ageb_mza_urbana_"+str(entidad)+"_cpv2020"):
            shutil.rmtree("datos/ageb_mza_urbana_"+str(entidad)+"_cpv2020")

In [8]:
#Pegar todos los archivos csv en una sola data frame
df=pd.DataFrame()
for entidad in range(1,33):
    if entidad<10:
        df_temp=pd.read_csv("datos/conjunto_de_datos_ageb_urbana_"+"0"+str(entidad)+"_cpv2020.csv",encoding='utf-8', low_memory=False)
    else:
        df_temp=pd.read_csv("datos/conjunto_de_datos_ageb_urbana_"+str(entidad)+"_cpv2020.csv",encoding='utf-8', low_memory=False)
    df=pd.concat([df,df_temp],axis=0)
    print("Se ha agregado la entidad "+str(entidad)," a la dataframe")

#Imprimir el número de registros con separadores de miles
print("Se concluyó la concatenación. El conjunto de datos cuenta con "+str(df.shape[1])+" columnas y "+str("{:,}".format(df.shape[0]))+" registros")
df.columns=df.columns.str.lower()
#Eliminar todos los archivos csv de la carpeta datos
#archivos=[x for x in os.listdir('datos') if x.endswith('.csv')]
#for archivo in archivos:
#    os.remove("datos/"+archivo)    

Se ha agregado la entidad 1  a la dataframe
Se ha agregado la entidad 2  a la dataframe
Se ha agregado la entidad 3  a la dataframe
Se ha agregado la entidad 4  a la dataframe
Se ha agregado la entidad 5  a la dataframe
Se ha agregado la entidad 6  a la dataframe
Se ha agregado la entidad 7  a la dataframe
Se ha agregado la entidad 8  a la dataframe
Se ha agregado la entidad 9  a la dataframe
Se ha agregado la entidad 10  a la dataframe
Se ha agregado la entidad 11  a la dataframe
Se ha agregado la entidad 12  a la dataframe
Se ha agregado la entidad 13  a la dataframe
Se ha agregado la entidad 14  a la dataframe
Se ha agregado la entidad 15  a la dataframe
Se ha agregado la entidad 16  a la dataframe
Se ha agregado la entidad 17  a la dataframe
Se ha agregado la entidad 18  a la dataframe
Se ha agregado la entidad 19  a la dataframe
Se ha agregado la entidad 20  a la dataframe
Se ha agregado la entidad 21  a la dataframe
Se ha agregado la entidad 22  a la dataframe
Se ha agregado la e

In [9]:
#Mostrar todos los nombres de las columnas
print(df.columns.tolist())

['entidad', 'nom_ent', 'mun', 'nom_mun', 'loc', 'nom_loc', 'ageb', 'mza', 'pobtot', 'pobfem', 'pobmas', 'p_0a2', 'p_0a2_f', 'p_0a2_m', 'p_3ymas', 'p_3ymas_f', 'p_3ymas_m', 'p_5ymas', 'p_5ymas_f', 'p_5ymas_m', 'p_12ymas', 'p_12ymas_f', 'p_12ymas_m', 'p_15ymas', 'p_15ymas_f', 'p_15ymas_m', 'p_18ymas', 'p_18ymas_f', 'p_18ymas_m', 'p_3a5', 'p_3a5_f', 'p_3a5_m', 'p_6a11', 'p_6a11_f', 'p_6a11_m', 'p_8a14', 'p_8a14_f', 'p_8a14_m', 'p_12a14', 'p_12a14_f', 'p_12a14_m', 'p_15a17', 'p_15a17_f', 'p_15a17_m', 'p_18a24', 'p_18a24_f', 'p_18a24_m', 'p_15a49_f', 'p_60ymas', 'p_60ymas_f', 'p_60ymas_m', 'rel_h_m', 'pob0_14', 'pob15_64', 'pob65_mas', 'prom_hnv', 'pnacent', 'pnacent_f', 'pnacent_m', 'pnacoe', 'pnacoe_f', 'pnacoe_m', 'pres2015', 'pres2015_f', 'pres2015_m', 'presoe15', 'presoe15_f', 'presoe15_m', 'p3ym_hli', 'p3ym_hli_f', 'p3ym_hli_m', 'p3hlinhe', 'p3hlinhe_f', 'p3hlinhe_m', 'p3hli_he', 'p3hli_he_f', 'p3hli_he_m', 'p5_hli', 'p5_hli_nhe', 'p5_hli_he', 'phog_ind', 'pob_afro', 'pob_afro_f', 'po

In [10]:
df

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,pobtot,pobfem,...,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
0,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la entidad,0000,0,1425607,728924,...,147818,359895,236003,174089,98724,70126,6021,15323,128996,1711
1,1,Aguascalientes,1,Aguascalientes,0,Total del municipio,0000,0,948990,486917,...,116647,251719,178619,130290,80951,56131,3299,7293,74227,731
2,1,Aguascalientes,1,Aguascalientes,1,Total de la localidad urbana,0000,0,863893,444725,...,112002,232793,169675,123670,77719,53589,2995,5984,63661,595
3,1,Aguascalientes,1,Aguascalientes,1,Total AGEB urbana,0017,0,2237,1137,...,11,625,189,352,46,74,15,23,391,*
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,170,87,...,*,53,13,33,5,5,0,*,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,2,*,...,*,*,*,*,*,*,*,*,*,*
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,0,0,...,0,0,0,0,0,0,0,0,0,0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0,...,0,0,0,0,0,0,0,0,0,0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,2,*,...,*,*,*,*,*,*,*,*,*,*


In [11]:
df= df.iloc[:, list(range(8)) + list(range(177, len(df.columns)))]
print("El conjunto de datos cuenta con "+str(df.shape[1])+" columnas y "+str("{:,}".format(df.shape[0]))+" registros")
df

El conjunto de datos cuenta con 61 columnas y 1,683,504 registros


,entidad,nom_ent,mun,nom_mun,loc,nom_loc,ageb,mza,vivtot,tvivhab,...,vph_telef,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic
0,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la entidad,0000,0,463972,386671,...,147818,359895,236003,174089,98724,70126,6021,15323,128996,1711
1,1,Aguascalientes,1,Aguascalientes,0,Total del municipio,0000,0,313256,266942,...,116647,251719,178619,130290,80951,56131,3299,7293,74227,731
2,1,Aguascalientes,1,Aguascalientes,1,Total de la localidad urbana,0000,0,286646,246259,...,112002,232793,169675,123670,77719,53589,2995,5984,63661,595
3,1,Aguascalientes,1,Aguascalientes,1,Total AGEB urbana,0017,0,1288,648,...,11,625,189,352,46,74,15,23,391,*
4,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,0017,1,82,54,...,*,53,13,33,5,5,0,*,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33839,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,18,3,1,...,*,*,*,*,*,*,*,*,*,*
33840,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,19,1,*,...,0,0,0,0,0,0,0,0,0,0
33841,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,20,0,0,...,0,0,0,0,0,0,0,0,0,0
33842,32,Zacatecas,58,Santa María de la Paz,1,Santa María de la Paz,0123,21,1,1,...,*,*,*,*,*,*,*,*,*,*


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683504 entries, 0 to 33843
Data columns (total 61 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   entidad      1683504 non-null  int64 
 1   nom_ent      1683504 non-null  object
 2   mun          1683504 non-null  int64 
 3   nom_mun      1683504 non-null  object
 4   loc          1683504 non-null  int64 
 5   nom_loc      1683504 non-null  object
 6   ageb         1683504 non-null  object
 7   mza          1683504 non-null  int64 
 8   vivtot       1683504 non-null  int64 
 9   tvivhab      1683504 non-null  object
 10  tvivpar      1683504 non-null  object
 11  vivpar_hab   1683504 non-null  object
 12  vivparh_cv   1683504 non-null  object
 13  tvivparhab   1683504 non-null  object
 14  vivpar_des   1683504 non-null  object
 15  vivpar_ut    1683504 non-null  object
 16  ocupvivpar   1683504 non-null  object
 17  prom_ocup    1683504 non-null  object
 18  pro_ocup_c   1683504 non